In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

In [2]:
df_nys_data = pd.read_csv("/Users/mannyb/Downloads/report_index.csv")

In [3]:
df_nys_data.head()

,County,Agency,Year,Months Reported,Index Total,Violent Total,Murder,Rape,Robbery,Aggravated Assault,Property Total,Burglary,Larceny,Motor Vehicle Theft,Region
0,Albany,Albany City PD,2021,12.0,3837.0,945.0,24.0,60.0,179.0,682.0,2892.0,368.0,2170.0,354.0,Non-New York City
1,Albany,Albany County Sheriff,2021,12.0,73.0,18.0,0.0,0.0,1.0,17.0,55.0,9.0,41.0,5.0,Non-New York City
2,Albany,Albany County State Police,2021,12.0,125.0,43.0,0.0,27.0,1.0,15.0,82.0,7.0,66.0,9.0,Non-New York City
3,Albany,Altamont Vg PD,2021,12.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,Non-New York City
4,Albany,Bethlehem Town PD,2021,12.0,329.0,22.0,1.0,5.0,4.0,12.0,307.0,27.0,269.0,11.0,Non-New York City


In [4]:
county_data = df_nys_data.groupby(['County','Year'], as_index =False).sum()
county_data["Year"] = pd.to_datetime(county_data["Year"],format='%Y')
county_data = county_data[['County','Year','Index Total','Violent Total','Property Total']]

/var/folders/f7/py50vmrj0gd2kwzqnc84nxrr0000gn/T/ipykernel_64490/1048671457.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  county_data = df_nys_data.groupby(['County','Year'], as_index =False).sum()


In [5]:
ny_sum = county_data.groupby(['Year'], as_index =False).sum()
index_crime_act = ny_sum["Index Total"].iloc[31]
violent_crime_act = ny_sum["Violent Total"].iloc[31]
property_crime_act = ny_sum["Property Total"].iloc[31]

/var/folders/f7/py50vmrj0gd2kwzqnc84nxrr0000gn/T/ipykernel_64490/3478787117.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ny_sum = county_data.groupby(['Year'], as_index =False).sum()


In [6]:
import warnings
warnings.filterwarnings('ignore')

county_list = county_data["County"].unique()
county_list = np.delete(county_list,np.where(county_list == "Saratoga"))
county_list = np.delete(county_list,np.where(county_list == "Westchester"))
violent_crime_pred = 0

for i in county_list:
    
    #print(i)
    data_large = county_data[county_data["County"]==i]
    data_index = data_large["Violent Total"]
    # Load the dataset and use the 'Close' column for the analysis
    data = data_index

    # Split the data into training and testing sets
    train_size = 30
    train, test = data[:train_size], data[train_size:]

    # Perform a grid search for the best ARIMA model
    best_aic = np.inf
    best_order = None
    best_model = None

    for p in range(3):
        for d in range(3):
            for q in range(3):
                try:
                    model = ARIMA(train, order=(p, d, q))
                    results = model.fit()
                    aic = results.aic

                    if aic < best_aic:
                        best_aic = aic
                        best_order = (p, d, q)
                        best_model = model

                except ValueError:
                    continue
    
    #print(f'County: {i}')
    # Print the best model's AIC and order
    #print(f'Best AIC: {best_aic:.2f}, Best order: {best_order}')
    
    train.index = data_large["Year"].iloc[:30]
    test.index = data_large["Year"].iloc[30:] 

    model = ARIMA(train, order=best_order)
    results = model.fit()
    predictions = results.predict(start=len(train), end=len(train) + len(test) - 1, dynamic=True)

    violent_crime_pred = violent_crime_pred + predictions.iloc[1]

    #mse = mean_squared_error(test, predictions)
    #print(f'Mean Squared Error: {mse:.2f}')

    #rmse = np.sqrt(mse)
    #print(f'Root Mean Squared Error: {rmse:.2f}')

    #mae = mean_absolute_error(test, predictions)
    #print(f'Mean Absolute Error: {mae:.2f}')

    #print('-----------------------------------')
    
saratoga = county_data[county_data["County"]=="Saratoga"]
westchester = county_data[county_data["County"]=="Westchester"]

violent_crime_pred = violent_crime_pred + saratoga["Violent Total"].iloc[29]
violent_crime_pred = violent_crime_pred + westchester["Violent Total"].iloc[29]

In [7]:
import warnings
warnings.filterwarnings('ignore')

county_list = county_data["County"].unique()
county_list = np.delete(county_list,np.where(county_list == "Saratoga"))
county_list = np.delete(county_list,np.where(county_list == "Westchester"))
property_crime_pred = 0

for i in county_list:
    
    #print(i)
    data_large = county_data[county_data["County"]==i]
    data_index = data_large["Property Total"]
    # Load the dataset and use the 'Close' column for the analysis
    data = data_index

    # Split the data into training and testing sets
    train_size = 30
    train, test = data[:train_size], data[train_size:]

    # Perform a grid search for the best ARIMA model
    best_aic = np.inf
    best_order = None
    best_model = None

    for p in range(3):
        for d in range(3):
            for q in range(3):
                try:
                    model = ARIMA(train, order=(p, d, q))
                    results = model.fit()
                    aic = results.aic

                    if aic < best_aic:
                        best_aic = aic
                        best_order = (p, d, q)
                        best_model = model

                except ValueError:
                    continue
    
    #print(f'County: {i}')
    # Print the best model's AIC and order
    #print(f'Best AIC: {best_aic:.2f}, Best order: {best_order}')
    
    train.index = data_large["Year"].iloc[:30]
    test.index = data_large["Year"].iloc[30:] 

    model = ARIMA(train, order=best_order)
    results = model.fit()
    predictions = results.predict(start=len(train), end=len(train) + len(test) - 1, dynamic=True)

    property_crime_pred = property_crime_pred + predictions.iloc[1]

    #mse = mean_squared_error(test, predictions)
    #print(f'Mean Squared Error: {mse:.2f}')

    #rmse = np.sqrt(mse)
    #print(f'Root Mean Squared Error: {rmse:.2f}')

    #mae = mean_absolute_error(test, predictions)
    #print(f'Mean Absolute Error: {mae:.2f}')

    #print('-----------------------------------')
    
saratoga = county_data[county_data["County"]=="Saratoga"]
westchester = county_data[county_data["County"]=="Westchester"]

property_crime_pred = property_crime_pred + saratoga["Property Total"].iloc[29]
property_crime_pred = property_crime_pred + westchester["Property Total"].iloc[29]

In [8]:
import warnings
warnings.filterwarnings('ignore')

county_list = county_data["County"].unique()
county_list = np.delete(county_list,np.where(county_list == "Saratoga"))
county_list = np.delete(county_list,np.where(county_list == "Westchester"))
index_crime_pred = 0

for i in county_list:
    
    #print(i)
    data_large = county_data[county_data["County"]==i]
    data_index = data_large["Index Total"]
    # Load the dataset and use the 'Close' column for the analysis
    data = data_index

    # Split the data into training and testing sets
    train_size = 30
    train, test = data[:train_size], data[train_size:]

    # Perform a grid search for the best ARIMA model
    best_aic = np.inf
    best_order = None
    best_model = None

    for p in range(3):
        for d in range(3):
            for q in range(3):
                try:
                    model = ARIMA(train, order=(p, d, q))
                    results = model.fit()
                    aic = results.aic

                    if aic < best_aic:
                        best_aic = aic
                        best_order = (p, d, q)
                        best_model = model

                except ValueError:
                    continue
    
    #print(f'County: {i}')
    # Print the best model's AIC and order
    #print(f'Best AIC: {best_aic:.2f}, Best order: {best_order}')
    
    train.index = data_large["Year"].iloc[:30]
    test.index = data_large["Year"].iloc[30:] 

    model = ARIMA(train, order=best_order)
    results = model.fit()
    predictions = results.predict(start=len(train), end=len(train) + len(test) - 1, dynamic=True)

    index_crime_pred = index_crime_pred + predictions.iloc[1]

    #mse = mean_squared_error(test, predictions)
    #print(f'Mean Squared Error: {mse:.2f}')

    #rmse = np.sqrt(mse)
    #print(f'Root Mean Squared Error: {rmse:.2f}')

    #mae = mean_absolute_error(test, predictions)
    #print(f'Mean Absolute Error: {mae:.2f}')

    #print('-----------------------------------')
    
saratoga = county_data[county_data["County"]=="Saratoga"]
westchester = county_data[county_data["County"]=="Westchester"]

index_crime_pred = index_crime_pred + saratoga["Index Total"].iloc[29]
index_crime_pred = index_crime_pred + westchester["Index Total"].iloc[29]

In [9]:
df_time_series = pd.DataFrame({"crime":["Violent Total","Property Total","Index Total"],
                               "timeseries":[violent_crime_pred,property_crime_pred,index_crime_pred]})

In [10]:
df_nys_estimation = pd.read_csv("/Users/mannyb/Downloads/Indicator_Tables_no_supp_no_LEOKA_88.csv")

In [11]:
df_nys_estimation_index_crime = df_nys_estimation[df_nys_estimation["indicator_name"].isin(["Violent Crime", "Property Crime",\
                                          "Murder and Non-negligent Manslaughter", "Rape", "Robbery", "Aggravated Assault",\
                                          "Burglary/B&E", "Larceny/Theft Offenses", "Motor Vehicle Theft"])]

df_nys_estimation_index_crime_summary = df_nys_estimation_index_crime[(df_nys_estimation_index_crime["estimate_type"] == "count") &\
                              (df_nys_estimation_index_crime["estimate_domain_1"] == "Offense count")]\
                              [["indicator_name", "estimate", "estimate_unweighted", "estimate_upper_bound", "estimate_lower_bound"]]
                            
df_nys_estimation_index_crime_summary["indicator_name"] = df_nys_estimation_index_crime_summary["indicator_name"].replace({
                                  "Violent Crime": "Violent Total",
                                  "Burglary/B&E": "Burglary",
                                  "Larceny/Theft Offenses": "Larceny",
                                  "Property Crime": "Property Total"
                              })

df_nys_estimation_index_crime_summary = df_nys_estimation_index_crime_summary.rename(columns = {
    "indicator_name": "crime"
})

In [12]:
df_nys_data_summary = df_nys_data[(df_nys_data["Year"] == 2021) &\
            (df_nys_data["Agency"] == "County Total")][["Index Total", "Violent Total", "Property Total",\
             "Murder", "Rape", "Robbery", "Aggravated Assault",\
             "Burglary", "Larceny", "Motor Vehicle Theft"]].sum(axis=0)\
             .reset_index() \
             .rename(columns = {
                 "index": "crime",
                 0: "nys_total"
             })

In [13]:
df2 = pd.merge(df_nys_estimation_index_crime_summary, df_nys_data_summary, on="crime", how="outer")

In [14]:
df = pd.merge(df2,df_time_series,on="crime",how="outer")

In [15]:
df = df.set_index("crime")

In [16]:
df.loc["Index Total", "estimate"] = df.loc["Violent Total", "estimate"] + df.loc["Property Total", "estimate"]
df.loc["Index Total", "estimate_unweighted"] = df.loc["Violent Total", "estimate_unweighted"] + df.loc["Property Total", "estimate_unweighted"]

In [17]:
df["diff_estimate"] = (abs(df['nys_total'] - df['estimate']))/(df['nys_total'])
df["diff_timeseries"] = (abs(df['nys_total'] - df['timeseries']))/(df['nys_total'])
df

,estimate,estimate_unweighted,estimate_upper_bound,estimate_lower_bound,nys_total,timeseries,diff_estimate,diff_timeseries
crime,,,,,,,,
Rape,1420.530702,432.0,1958.666098,882.395306,6213.0,NaN,0.771362,NaN
Violent Total,20202.815789,7464.0,31513.366511,8892.265068,76229.0,93905.044274,0.734972,0.231881
Burglary,15523.881579,5406.0,24573.598267,6474.164891,27129.0,NaN,0.427775,NaN
Larceny,114465.517544,39977.0,180474.092856,48456.942232,218129.0,NaN,0.475239,NaN
Property Total,140832.311404,49595.0,223183.227308,58481.395499,267128.0,350368.382144,0.472791,0.311612
Index Total,161035.127193,57059.0,NaN,NaN,343357.0,472005.590169,0.530998,0.374679
Murder,NaN,NaN,NaN,NaN,880.0,NaN,NaN,NaN
Robbery,NaN,NaN,NaN,NaN,18050.0,NaN,NaN,NaN
Aggravated Assault,NaN,NaN,NaN,NaN,51086.0,NaN,NaN,NaN
